In [1]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

In [2]:
from kinematicsrobotics.datahandler import Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics
from kinematicsrobotics.kinematics import Robo,Spacemapping

## Dataset

In [3]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe( r'src\data\ready\dataset-radius-1cm.csv')
dataset

,theta_1,theta_2,theta_3,theta_4,theta_5,p_x,p_y,p_z,roll,pich,yaw
0,0,0,12,0,0,53.213314,-2.477121e-15,2.515179,3.141593e+00,-1.361357,2.752040e-16
1,0,12,12,0,0,53.606657,-1.789652e-15,13.742410,2.378531e+00,-1.570796,7.630613e-01
2,0,12,24,0,0,52.819970,-3.124530e-15,6.257590,3.141593e+00,-1.361357,2.371518e-16
3,0,12,24,12,0,53.213314,-2.915404e-15,10.000000,2.356194e+00,-1.570796,7.853982e-01
4,0,12,36,12,0,51.657132,-4.201082e-15,2.678740,3.141593e+00,-1.361357,2.500234e-16
...,...,...,...,...,...,...,...,...,...,...,...
6884,120,120,120,60,0,-9.000000,1.558846e+01,41.176915,-2.026424e-16,-0.523599,-1.047198e+00
6885,120,120,120,72,0,-7.281153,1.261133e+01,42.707475,-1.899993e-16,-0.314159,-1.047198e+00
6886,120,120,120,84,0,-5.440756,9.423666e+00,43.489851,-1.843716e-16,-0.104720,-1.047198e+00
6887,120,120,120,96,0,-3.559244,6.164791e+00,43.489851,-1.843716e-16,0.104720,-1.047198e+00


In [4]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4'],
                           path_data_split = r'src\data\data-split'
                        )

OSError: Cannot save file into a non-existent directory: 'c:\Users\je7560\Downloads\matheus\kinematics-robotics\test\src\data\data-split'

In [5]:
x_train, x_test, y_train, y_test = data.data_train_test

In [14]:
from pandas import DataFrame
DataFrame({'labels':data._x_labels,
           'Média': data._scaler_x.mean_, 
           'Desvio Padrão': data._scaler_x.scale_})

,x_labels,Média,Desvio Padrão
0,p_x,7.831121,19.759515
1,p_y,13.521475,21.855324
2,p_z,34.952026,16.667249
3,roll,0.530999,1.593114
4,pich,-0.471452,1.013631
5,yaw,-0.962523,1.689215


In [16]:
DataFrame({'labels':data._y_labels,
           'Média': data._scaler_y.mean_, 
           'Desvio Padrão': data._scaler_y.scale_})

,y_labels,Média,Desvio Padrão
0,theta_1,60.506452,36.280626
1,theta_2,84.392903,31.580442
2,theta_3,60.596129,34.592720
3,theta_4,37.875484,37.447529


In [12]:
data._scaler_y.mean_

array([60.50645161, 84.39290323, 60.59612903, 37.87548387])

In [15]:
data._scaler_y.scale_

array([36.28062609, 31.58044231, 34.59271951, 37.44752861])

## Modelo

In [6]:
mlp = Model.mlp_regressor(hidden_layer_sizes=(300,300), EPOCHS_NOCHANGE = 20, activation = 'tanh')

## Estimação de Parâmatros

In [19]:
# Estimação de hiperparâmetros
history = ext.dataframe(r'src\data\ready\history.csv')

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

In [8]:
mlp.set_model(**params)

## Treino

In [7]:
mlp.fit(x=x_train,y=y_train)

## Metricas

In [8]:
metrica = Metrics(model = mlp, 
                  preprocessing = data)

In [9]:
metrica.mse(mlp.predict(x =x_train), y_train)

array([0.00094481, 0.00339415, 0.00850638, 0.00353521])

### Erro juntas

In [12]:
metrica.predict_joint(x = x_train)

array([[ 24.29214127, 108.73390695,  97.13105509,  34.66951297],
       [ 82.44981689,  46.75644653,  81.5193677 ,  47.2624096 ],
       [ 97.96515396,  83.53616809,  71.61121945,  11.82312133],
       ...,
       [ 55.24124326, 108.59305289,  25.3201193 ,  84.28555467],
       [105.18169889, 117.38684258,   8.75493542,  59.17692366],
       [ 93.25165388,  71.39439563,  69.17233769,  45.66112949]])

In [13]:
metrica.mse_joint(x = x_train, y = y_train)

array([ 1.25035967,  3.40159594, 10.17588975,  4.96185004])

### Erro cartesiano

In [14]:
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],
        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]
]

robo = Robo("Robo", Elos)



In [15]:
metrica.robotic(robo = robo)

In [17]:
metrica.predict_operacional(x = x_train)

,p_x,p_y,p_z
0,22.142282,9.993968,43.674056
1,5.872351,44.305035,16.744544
2,-5.004314,35.765356,38.853806
3,-12.110327,49.579186,25.949554
4,10.592818,21.922441,49.813411
...,...,...,...
6195,26.463566,21.875547,38.676459
6196,-7.666197,16.802817,41.365477
6197,-12.091141,-17.423622,48.814577
6198,8.281923,-30.521067,46.840345


In [16]:
metrica.mse_operacional(x = x_test)

array([0.35811319, 0.32361371, 0.50799457])